# Riconoscere Tumori al Seno Maligni
Mettiamo alla prova la nostra rete neurale su un problema più impegnativo, riconoscere tumori al seno maligni partendo da esami agobiopsici. A tal proposito utilizzere il [Bostob Breast Cancer Dataset](https://www.kaggle.com/uciml/breast-cancer-wisconsin-data). Per caricare il file da un csv possiamo utilizzare Pandas, una popolare libreria Python per l'analisi dati.

In [1]:
import pandas as pd

CSV_URL = "https://raw.githubusercontent.com/ProfAI/tutorials/master/Come%20Creare%20una%20Rete%20Neurale%20da%20Zero/breast_cancer.csv"

breast_cancer = pd.read_csv(CSV_URL)

Il risultato sarà un DataFrame, una struttura dati che Pandas usa per rappresentare dati tabulari, possiamo avere una preview del suo contenuto usando il metodo .head().
Il nostro dataset contiene in totale 563 righe (e quindi esempi) e 31 colonne, cioè 30 features e un target, che è la colonna “malignant”.
Estraiamo features e target in array numpy.

In [2]:
X = breast_cancer.drop("malignant", axis=1).values
y = breast_cancer["malignant"].values

Ora dobbiamo dividere ogni array in due array distinti, uno per l’addestramento e uno per il test. Questa divisione serve per poter verificare le reali capacità predittive del modello, testandolo su dati che non ha già visto durante la fase di addestramento.
L’overfitting di cui abbiamo accennato all’inizio è la condizione in cui il modello memorizza i dati di addestramento piuttosto che apprendere da essi, avere un set di test separato ci permette di identificare questa situazione.
Creiamo una funzione train_test_split per eseguire questa divisione:

In [4]:
import numpy as np

def train_test_split(X, y, test_size=0.3, random_state=None):

      if(random_state!=None):
        np.random.seed(random_state)

      n = X.shape[0]

      test_indices = np.random.choice(n, int(n*test_size), replace=False) # selezioniamo gli indici degli esempi per il test set

      # estraiamo gli esempi del test set
      # in base agli indici

      X_test = X[test_indices]
      y_test = y[test_indices]

      # creiamo il train set
      # rimuovendo gli esempi del test set
      # in base agli indici

      X_train = np.delete(X, test_indices, axis=0)
      y_train = np.delete(y, test_indices, axis=0)

      return (X_train, X_test, y_train, y_test )


X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.3)

Abbiamo assegnato il 30% degli esempi del dataset al set di test, quindi abbiamo 395 esempi per l’addestramento e 168 per il test, sono un po’ pochi per l’addestramento di una rete neurale, ma trattandosi di un modello con un solo strato nascosto possono andare bene.
E’ buona norma portare le features in un range di valori comune, questo può velocizzare anche di tanto la fase di addestramento.
Utilizziamo la normalizzazione, che si esegue sottraendo il valore minore e dividendo per la differenza tra il valore maggiore e il valore minore:

$$ X_{norm} = \frac{X-X_{min}}{X_{max}-X_{min}} $$

Ricorda che dobbiamo sempre applicare le stesse trasformazioni ai dati di addestramento, a quelli di test, e in generale a tutti quelli che daremo in pasto alla nostra rete neurale, quindi calcoliamo massimo e minimo sul set di addestramento e usiamo questi valori per la normalizzazione di entrambi gli array.

In [5]:
X_max = X_train.max(axis=0)
X_min = X_train.min(axis=0)

X_train = (X_train - X_min)/(X_max-X_min)
X_test = (X_test - X_min)/(X_max-X_min)

Perfetto ! Adesso importiamo la nostra classe *NeuralNetwork* e creiamo la nostra rete con 10 nodi sullo strato nascosto, addestriamola sul set di addestramento per 500 epoche e valutiamola sul set di test.

In [12]:
model = NeuralNetwork()
model.fit(X_train, y_train, epochs=500, lr=0.01)
model.evaluate(X_test, y_test)

(0.9642857142857143, 0.08487508700240488)

Dato che i pesi vengono inizializzati a valori casuali il risultato può lievemente variare tra diverse esecuzioni della rete.

## Bene ! E ora ?
Mettiamo caso di ricevere i risultati di 6 nuovi esami radiografici, le features estratte da questi ci vengono consegnate all’interno di un file csv, carichiamolo con pandas, estraiamolo le features e normalizziamole

In [6]:
exams_df = pd.read_csv("https://raw.githubusercontent.com/ProfAI/tutorials/master/Come%20Creare%20una%20Rete%20Neurale%20da%20Zero/exam%20results.csv")

X_new = exams_df.values
X_new = (X_new - X_min)/(X_max-X_min)

Ora utilizziamo il metodo predict per classificare i risultati di tali esami, in modo da identificare eventuali tumori maligni, ottenendo anche la probabilità.

In [10]:
y_pred, y_proba = model.predict(X_new, return_proba=True)

Stampiamo il risultato

In [11]:
classes = ["benigno", "maligno"]

for i, (pred, proba) in enumerate(zip(y_pred, y_proba)):
  print("Risultato %d = %s (%.4f)" % (i+1, classes[int(pred)], proba))

Risultato 1 = benigno (0.0000)
Risultato 2 = maligno (0.9974)
Risultato 3 = maligno (0.9991)
Risultato 4 = benigno (0.0156)
Risultato 5 = maligno (0.9652)
Risultato 6 = benigno (0.0165)


Quando la probabilità associata non è alta andrebbero eseguiti ulteriori esami di verifica, specialmente nel caso di un tumore classificato come benigno, dato che classificare erroneamente un tumore maligno come benigno è molto più grave che classificare un tumore benigno come maligno. Come abbiamo visto in questo tutorial, in questi casi è opportuno utilizzare anche la matrice di confusione come metrica per valutare il modello.